<a href="https://colab.research.google.com/github/GOPIKA-S-S/NLP_Lab/blob/main/Gopika_518_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Example documents or paragraphs
doc1 = "The sun rises in the east and sets in the west, casting a warm glow over the landscape. Birds chirp as they soar through the clear blue skies, and a gentle breeze rustles the leaves on the trees."
doc2 = "As dawn breaks, the sun ascends over the horizon, illuminating the terrain with its golden rays. The melodious symphony of bird songs fills the air, and a tranquil zephyr dances through the branches, causing the foliage to sway rhythmically."

# Function to calculate Cosine Similarity
def calculate_cosine_similarity(doc1, doc2):
    vectorizer = CountVectorizer().fit_transform([doc1, doc2])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0, 1]

# Function to calculate Jaccard Similarity
def calculate_jaccard_similarity(doc1, doc2):
    words_doc1 = set(doc1.split())
    words_doc2 = set(doc2.split())
    intersection = len(words_doc1.intersection(words_doc2))
    union = len(words_doc1.union(words_doc2))
    return intersection / union

# Calculate Cosine Similarity
cosine_sim = calculate_cosine_similarity(doc1, doc2)
print("Cosine Similarity:", cosine_sim)

# Calculate Jaccard Similarity
jaccard_sim = calculate_jaccard_similarity(doc1, doc2)
print("Jaccard Similarity:", jaccard_sim)


Cosine Similarity: 0.6790640976965928
Jaccard Similarity: 0.12280701754385964


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df=pd.read_csv("/content/IMDB Dataset.csv")

In [4]:
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
df.shape

(50000, 2)

In [7]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [8]:
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Preprocessing function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply preprocessing to 'review' column
df['review'] = df['review'].apply(preprocess_text)

# Split into features and target
X = df['review'].values
y = df['sentiment'].values

# Split the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [9]:
# Tokenization and padding
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Check the shape of the data
print("X_train shape:", X_train_pad.shape)
print("X_val shape:", X_val_pad.shape)
print("X_test shape:", X_test_pad.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)

X_train shape: (35000, 100)
X_val shape: (7500, 100)
X_test shape: (7500, 100)
y_train shape: (35000,)
y_val shape: (7500,)
y_test shape: (7500,)


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Train the Multinomial Naive Bayes classifier
bayes_classifier = MultinomialNB()
bayes_classifier.fit(X_train_tfidf, y_train)

# Predictions on validation set
y_pred_val = bayes_classifier.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_pred_val)
print("Validation Accuracy (Bayesian):", val_accuracy)

# Predictions on test set
y_pred_test = bayes_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy (Bayesian):", test_accuracy)


Validation Accuracy (Bayesian): 0.8494666666666667
Test Accuracy (Bayesian): 0.8569333333333333


In [11]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

# Convert target variable
y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)
y_test = y_test.astype(np.float32)


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Build the RNN model
model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_rnn.add(SimpleRNN(units=128, dropout=0.2))  # Adding dropout for regularization
model_rnn.add(Dense(units=1, activation='sigmoid'))

model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the RNN model
history_rnn = model_rnn.fit(X_train_pad, y_train, validation_data=(X_val_pad, y_val), epochs=5, batch_size=128)

# Evaluate the RNN model
val_loss_rnn, val_accuracy_rnn = model_rnn.evaluate(X_val_pad, y_val)
print("Validation Accuracy (RNN):", val_accuracy_rnn)

test_loss_rnn, test_accuracy_rnn = model_rnn.evaluate(X_test_pad, y_test)
print("Test Accuracy (RNN):", test_accuracy_rnn)


Epoch 1/5
274/274 [==============================] - 46s 163ms/step - loss: 0.5464 - accuracy: 0.7018 - val_loss: 0.4401 - val_accuracy: 0.8035
Epoch 2/5
274/274 [==============================] - 45s 163ms/step - loss: 0.3382 - accuracy: 0.8597 - val_loss: 0.3775 - val_accuracy: 0.8407
Epoch 3/5
274/274 [==============================] - 44s 162ms/step - loss: 0.2386 - accuracy: 0.9058 - val_loss: 0.4345 - val_accuracy: 0.8297
Epoch 4/5
274/274 [==============================] - 47s 173ms/step - loss: 0.1546 - accuracy: 0.9427 - val_loss: 0.5203 - val_accuracy: 0.8155
Epoch 5/5
235/235 [==============================] - 3s 13ms/step - loss: 0.6172 - accuracy: 0.8141
Validation Accuracy (RNN): 0.81413334608078
235/235 [==============================] - 3s 14ms/step - loss: 0.5923 - accuracy: 0.8243
Test Accuracy (RNN): 0.8242666721343994


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Build the LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_lstm.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))  # Adding dropout for regularization
model_lstm.add(Dense(units=1, activation='sigmoid'))

model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the LSTM model
history_lstm = model_lstm.fit(X_train_pad, y_train, validation_data=(X_val_pad, y_val), epochs=5, batch_size=128)

# Evaluate the LSTM model
val_loss_lstm, val_accuracy_lstm = model_lstm.evaluate(X_val_pad, y_val)
print("Validation Accuracy (LSTM):", val_accuracy_lstm)

test_loss_lstm, test_accuracy_lstm = model_lstm.evaluate(X_test_pad, y_test)
print("Test Accuracy (LSTM):", test_accuracy_lstm)


Epoch 1/5
274/274 [==============================] - 213s 762ms/step - loss: 0.4230 - accuracy: 0.7975 - val_loss: 0.3317 - val_accuracy: 0.8571
Epoch 2/5
274/274 [==============================] - 207s 755ms/step - loss: 0.2828 - accuracy: 0.8849 - val_loss: 0.3354 - val_accuracy: 0.8521
Epoch 3/5
274/274 [==============================] - 208s 758ms/step - loss: 0.2270 - accuracy: 0.9100 - val_loss: 0.3465 - val_accuracy: 0.8479
Epoch 4/5
274/274 [==============================] - 208s 758ms/step - loss: 0.1805 - accuracy: 0.9310 - val_loss: 0.3942 - val_accuracy: 0.8476
Epoch 5/5
235/235 [==============================] - 9s 40ms/step - loss: 0.4404 - accuracy: 0.8453
Validation Accuracy (LSTM): 0.8453333377838135
235/235 [==============================] - 9s 36ms/step - loss: 0.4467 - accuracy: 0.8472
Test Accuracy (LSTM): 0.8471999764442444
